In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np
import random
import time
from tqdm import tqdm
from itertools import combinations
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib as mpl
import datetime
import math
from scipy.stats import norm, kurtosis, mode
cmap = plt.get_cmap('tab10')
from scipy.stats import describe
import seaborn as sns
from matplotlib.pyplot import figure
import pickle

In [23]:
def one_simulation(p_out,G_input, N_walkers,initial_node_with_walkers,run_time):
    end_time_list = []
    r_square_re = []
    for iter_num in range(run_time):
        #G = G_ba.copy()
        G = G_input.copy()
        N_nodes = list(G.nodes())


        ### Asign the walker
        nx.set_node_attributes(G, [0],'history')
        initial_walkers = random.sample(N_nodes,initial_node_with_walkers)
        for i in initial_walkers:
            G.nodes[i]['history'] = [N_walkers / initial_node_with_walkers]
        
        degree_of_initial = [G.degree(k) for k in initial_walkers]

        # the x (expected final distribution of walkers) of the linear regression
        walker_per_degree = N_walkers/sum([j for (i,j) in G.degree])
        degree_list = np.array([j*walker_per_degree for (i,j) in G.degree])
        degree_list = degree_list.reshape(-1,1)
        
        
        # Run the simulation model until the R-square over 0.99
        linear_score = [0]
        end_time = 0
        while linear_score[-1] <0.99:
            end_time += 1
            for nodes in G.nodes:
                tem_out_list = [n for n in G.neighbors(nodes)]
                tem_in_node = 0
                for neigh_node in tem_out_list:
                    tem_in_node += G.nodes[neigh_node]['history'][end_time-1] * p_out / len([n for n in G.neighbors(neigh_node)])
                G.nodes[nodes]['history'] = G.nodes[nodes]['history']+[tem_in_node + G.nodes[nodes]['history'][end_time-1] * (1-p_out)]
            ## Run the linear regression
            check_list = []
            for nodes in G.nodes:
                check_list.append(G.nodes[nodes]['history'][-1])
            reg = LinearRegression().fit(degree_list, check_list)
            linear_score.append(round(reg.score(degree_list, check_list),4))
        # record the end time of each simulation    
        end_time_list.append(end_time)
        
    return end_time_list


In [24]:
p_out = 0.4
end_time_all_ER = []
end_time_all_BA = []
N_walkers = 10000
initial_node_with_walkers = 4
run_time = 100

for _ in tqdm(range(90,100)): ## 100 graph instances
    
    G_BA_total = nx.read_gpickle('data/4b-BA-instance-' + str(_) + '.pickle')
    G_ER_total = nx.read_gpickle('data/4b-ER-instance-' + str(_) + '.pickle')

    Gcc = sorted(nx.connected_components(G_BA_total), key=len, reverse=True)
    G_BA = G_BA_total.subgraph(Gcc[0])

    Gcc = sorted(nx.connected_components(G_ER_total), key=len, reverse=True)
    G_ER = G_ER_total.subgraph(Gcc[0])

    end_time_list_ER = one_simulation(p_out,G_ER, N_walkers, initial_node_with_walkers, run_time)
    end_time_list_BA = one_simulation(p_out,G_BA, N_walkers, initial_node_with_walkers, run_time)
    
    end_time_all_ER.append(end_time_list_ER)
    end_time_all_BA.append(end_time_list_BA)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [09:39<00:00, 57.96s/it]


In [7]:
#np.save('data/4b-end_time_all_ER_0_89_simulations_p_out' + str(p_out) + '.npy', np.array(end_time_all_ER))
#np.save('data/4b-end_time_all_BA_0_89_simulations_p_out_' + str(p_out) + '.npy', np.array(end_time_all_BA))

In [25]:
np.save('data/4b-end_time_all_ER_90_100_simulations_p_out' + str(p_out) + '.npy', np.array(end_time_all_ER))
np.save('data/4b-end_time_all_BA_90_100_simulations_p_out_' + str(p_out) + '.npy', np.array(end_time_all_BA))